# Assignment im Kurs Machine Learning Fundamentals

### Projekt Teilnehmer
* Anna Stöhrer da Silva
* Bernice Fabich
* Jan Schneeberg
* Niklas Elsässer

### Dozent
* Ruben Nuredini

## Abstract
Im Rahmen der vorliegenden Untersuchung wird die Leistungsfähigkeit der Latent Dirichlet Allocation Method (LDA) anhand eines umfangreichen News-Category-Datasets evaluiert. Dabei wird die reguläre LDA-Methode mit einem verbesserten Ansatz, LDAPrototype genannt, verglichen.

# Inhaltsverzeichnis
- [Theoretische Grundlagen](#theoretische-grundlagen)
  - [Datenvorbereinigung](#Datenvorbereinigung)
  - [LDA Modell](#step-4-das-lda-model-trainieren)
  - [PyLDAvis](#step-4-visualisierung-mit-pyldavis)
- [Improved LDA](https://github.com/NiklasElsaesser/bug-free-fishstick/blob/main/Improved_LDA.ipynb) (Separates Notebook)
- [Benchmarks & Visualizations](https://github.com/NiklasElsaesser/bug-free-fishstick/blob/main/Benchmarks%26Visualizations.ipynb) (Separates Notebook)
- [Literaturverzeichnis](#literaturverzeichnis)

# Theoretische Grundlagen
Durch die Digitalisierung steigt die Anzahl digitaler Texte drastisch an, sodass diese kaum noch manuell zu erfassen sind. So beheimatet zum Beispiel das Deutsche Textarchiv (digitale Textsammlung) um die 145 Millionen Texte.
Diese Masse an Texten muss verwaltet werden, um Texte zu finden und abzubilden. Des Weiteren müssen sie von Computern ausgewertet werden können, da dies manuell nicht machbar ist.
Im Jahr 2007 beschäftigte sich Franco Moretti mit seinem Forschungsprogramm damit, tausend Werke der Weltliteratur gleichzeitig miteinander zu vergleichen. Möglich war dies aber nur mit (text-) statischen Verfahren und computergestützten Visualisierungen. (Heyer, Wiedemann, Niekler 2020, S.351-353)

## Topic Modeling
Topic Modelling ist eine bekannte Klasse dieser statischen Verfahren. Hierbei können große Daten-, Textmengen automatisch in Themenbereiche aufgeteilt werden. Es wird davon ausgegangen, dass jede Wortform zu einem Themenbereich (Topic) gehört. Durch die Untersuchung der Verteilung von zusammengehörigen Wortformen in einem Text sollen die Themenbereiche dieses Textes abgeleitet werden können.
Hiermit ist das Klassifizieren und Zusammenfassen von Texten und deren Abschnitten möglich. (Heyer, Wiedemann, Niekler 2020, S.351-353)

## Stärken und Schwächen
Stärken sind demnach die Unterteilung von großen Texten in Themen (Topics), sowie die Tatsache, dass dies voll automatisch geschieht.
Nachteil ist, dass Syntax-Strukturen, Regelmäßigkeiten der Aussagen und weitere Informationen nicht aus dem Topic Modelling resultieren.
Dennoch kann Topic Modelling als Vorarbeit für die weitergehenden Untersuchungen genutzt werden. (Heyer, Wiedemann, Niekler 2020, S.351-353)


# Simple LDA
## Datenvorbereinigung
### Step 0: Libraries importieren

In [149]:
# The Basics
import pandas as pd
import numpy
from pprint import pprint

In [150]:
# The Preprocessing
import re
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/niklaselsasser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niklaselsasser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niklaselsasser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [151]:
# The Modell
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

### Step 1: Das Dataframe (be)laden

In [153]:
#code necessary when running in colab
import json
import pandas as pd

dataset_path = "News_Category_Dataset_v3.json"

with open(dataset_path, 'r') as file:
    news_data = [json.loads(line) for line in file]

df = pd.DataFrame(news_data)

In [154]:
#code necessary when running locally
df = pd.read_json("News_Category_Dataset_v3.json", lines=True)

### Step 2: Data Cleansing

In [155]:
df.drop(columns=["link", "authors", "date"], inplace=True)
df.head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


Auf den ersten Blick können wir erkennen, dass die Kategorie "U.S. NEWS" eher ein mix aus unterschiedlichen Themen ist.
<br>Das könnte später unseren Algorithmus verwirren, weshalb wir Kategorien welche aus unterschiedlichen Themen bestehen, entfernen wollen.

In [156]:
df.category.unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

Kategorien welche wahrscheinlich mehrere Themen umfassen werden erfasst. Im unteren Sample können wir erkennen, dass unsere These korrekt ist.

In [157]:
df[df["category"].isin(["U.S. NEWS", "WORLD NEWS", "WEIRD NEWS", "THE WORLDPOST", "WORLDPOST"])].sample(5, random_state=0)

,headline,category,short_description
4996,93-Year-Old Woman Goes Viral When She Tells In...,WEIRD NEWS,"On Monday, Coors Light dropped off 150 cans of..."
6399,Shooting At Brooklyn Community Event Leaves 1 ...,U.S. NEWS,New York City Mayor Bill de Blasio vowed to “d...
74496,Woman Accused Of Commandeering Bus After Drive...,WEIRD NEWS,She didn't get very far.
24611,Now I Know What Fear Is,WORLD NEWS,"Everywhere I looked on my way home, someone wa..."
127331,Reinventing Europe Along These 7 Points,WORLDPOST,"In its present form, the EU is weak, particula..."


Was auch ins Auge fällt, ist Zeile `74496`. Ihre short_description ist zu kurz um daraus einen wertvollen Einblick zu erhalten. Das merken wir uns für später ;)

In [158]:
# Removal of useless categories
df = df[~df["category"].isin(["U.S. NEWS", "WORLD NEWS", "WEIRD NEWS", "THE WORLDPOST", "WORLDPOST"])]

In [159]:
unique_categories = len(df.category.unique())
print(f"{unique_categories=}")

unique_categories=37


Wir haben die alten Kategorien entfernt und sind jetzt bei schlanken 37 Kategorien.
<br>Als nächstes bereiten wir den Text aus der Überschrift und der Beschreibung etwas auf.
<br>Dazu entfernen wir alle Interpunktierung, Stoppwörter, Sonderzeichen und sorgen dafür, dass alle Buchstaben klein geschrieben sind. Diese Datenbereinigung, sowie die Erstellung des Gensim-Dictionary wird im verbesserten LDA ausführlicher beschrieben.

In [160]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

Ein Beispiel wie diese Textaufbereitung funktiniert und was sie zurückgibt, findet sich in der unteren Zelle

In [161]:
print(preprocess_text("This is a very important text mentioning the President, the Health Department and Germany"), sep=" ")

['important', 'text', 'mentioning', 'president', 'health', 'department', 'germany']


In [162]:
# Apply preprocessing to the headlines and short descriptions
df['processed_text'] = df.apply(lambda row: preprocess_text(row['headline'] + ' ' + row['short_description']), axis=1)

# Filter out empty processed_text
processed_df = df[df['processed_text'].apply(len) > 0]

# Inspect the processed text
print(processed_df[['category', 'processed_text']].head())

          category                                     processed_text
2           COMEDY  [23, funniest, tweet, cat, dog, week, sept, 17...
3        PARENTING  [funniest, tweet, parent, week, sept, 1723, ac...
8   CULTURE & ARTS  [new, documentary, capture, complexity, child,...
13            TECH  [twitch, ban, gambling, site, streamer, scam, ...
16  CULTURE & ARTS  [reboot, clever, navelgazey, look, inside, tv,...


In [163]:
# Overview over preprocessed text
processed_df[['short_description', 'processed_text']].sample(5)

,short_description,processed_text
2965,The country singer paid tribute to Elvis Presl...,"[eric, church, get, covid19, vaccine, shot, bi..."
167389,Long was last rumored to be dating actress Kat...,"[justin, long, describes, wedding, test, hed, ..."
66615,"""African Americans will disproportionately bea...","[federal, court, rule, ohio, voting, cutback, ..."
38088,Drivers in some regions of the country display...,"[distracted, driver, safest, driver, region, c..."
14936,"""Yeah but at least I suck while singing our [c...","[pink, clap, back, troll, mocked, national, an..."


In [164]:
# Extract just the processed texts for future use
processed_texts = [doc["processed_text"] for _, doc in processed_df.iterrows()] 

### Step 3: Hilfsmittel für das LDA Model erstellen
Um das LDA Model später geordnet zu erzeugen, erstellen wir zwei Hilfsmittel.
### Dictionary: ID zu Wort
Corpora weißt jeden Wort eine eindeutige ID zu, mit welcher es später wieder erkannt werden kann.
<br>Dies hilft bei der schnellen und effizienten Verarbeitung der Wörter

### Liste: Korpus
Der Korpus listet für alle Dokumente einen Vektor welchen wir als "Bag of Words" bezeichnen.
<br>Er enthält alle Wörter und ihre Häufigkeit in dem Dokument. Über diesen Vektor, kann dem Dokument eine Bedeutung zugewiesen werden.

In [165]:
id2word = corpora.Dictionary(processed_texts)

corpus = [id2word.doc2bow(text) for text in processed_texts]

## LDA Modell
### Step 4: Das LDA Model trainieren
Das Latent Dirichlet Allocation (Latente Dirichlet Zuordnung (LDA)) Verfahren ist ein 3-stufiges Bayesian Model, um Topic Modeling durchzuführen (Blei et al.(2003)). Das Verfahren besteht aus den Teilen:
* **Latent:** Die Themen des Korpuses sind verborgen und müssen aus den Dokumenten und deren Wortverteilung abgeleitet werden (Blei et al. (2003), S. 1007).
* **Dirichlet:** Eine Verteilung, basierend auf dem Bayesian Modell, welche die Prioriverteilung (Anfangswahrscheinlichkeit) für die Themenverteilung in den Dokumenten darstellt und die Parameter beeinflussen wie stark bestimmte Themen in einem Dokument vertreten sind (Blei et al. (2003), S. 1007).
* **Allocation:** Die Prozess der Zuweisung und Verteilung beschreibt wie Wörter den verschiedenen Themen zugeordnet werden und wie die Themen in einem Dokument verteilt sind. Allocation beschreibt wie das Modell Wörter in einem Dokument aufgrund ihrer Wahrscheinlichkeit verschiedenen latenten Themen zugeordnet sind. Die Themen - Wörter verteilung in einem Dokument bildet die Grundlage der Analyse (Blei et al. (2003), S. 1007).

LDA nimmt an das es eine Themenverteilung für jeden Text gibt, welche berechnet wird indem ein Thema aus allen verfügbaren Themen $T={T_1,…,T_K}$ jedem Token ($≈$ Wort) eines Textes zugeordnet wird. Der Nutzer definiert dabei die Anzahl der modellierten Themen $K∈N$. Der Text (oder das Dokument) wird als Corpus bezeichnet und besteht aus Texten $M$ sowie Themen $T$ (Rieger at al. (2020 Juni), S. 120); Blei et al. (2003), S. 997).
Dabei ist $N^m$ die Größe des Textes und $W={W,…,W_V}$ die Menge der Wörter wobei $V∈N$ die Größe des implizierten Vokabulars (die Menge aller eindeutigen Wörter) ist(Rieger et al. (2020 Juni), S. 120).

<img src="visuals/page1.png" alt="p1" width="250" />
<br>
<img src="visuals/page2.png" alt="p2" width="250" />

Somit besteht ein $Dokument$ aus:
$$
D^{(m)} = (W_1^{(m)}, \dots, W_{N^{(m)}}^{(m)}), \quad W_n^{(m)} \in W; \quad n = 1, \dots, N^m
$$
Die Themenzuweisung für einen Text $m$ ist wie folgt:
$$
T^{(m)} = (T^{(m)}, \dots, T_{N^{(m)}}^{(m)}), \quad T_n^{(m)} \in T
$$
Jede Themenzuweisung $T_n^{(m)}$ hängt mit einem Token $W_n^{(m)}$ aus dem Text $m$ zusammen.

![page3.png](visuals/page3.png)

Um das Wahrscheinlichkeitsmodell für LDA aufzustellen sind folgende Definitionen und Annahmen notwendig: $n_k^{(mv)};k=1,…,K;v=1,…,V$ als Anzahl zugewiesener Wörter $v$ in Text $m$ zugehörig zu Thema $k$, dadurch lässt sich die Summe der Wörter $v$  in Thema $k$ über alle Dokumente $D$ mit $n_k^{(*v)}$ bestimmen. Wenn $w_k$ die Vektoren der Wortanzahl für $k=1,…,K$ Themen ist, dann lässt sich mit diesen Definitionen das Modell wie folgt aufstellen:

$$
W_n^{(m)} \mid T_n^{(m)}, \phi_k \sim \text{Discrete}(\phi_k), \quad \phi_k \sim \text{Dirichlet}(\eta)
$$

$$
T_n^{(m)} \mid \theta_m \sim \text{Discrete}(\theta_m), \quad \theta_m \sim \text{Dirichlet}(\alpha)
$$


Die Dirichlet Verteilungs Hyperparameter $α$ und $η$ müssen vom Nutzer deklariert werden. Da normalerweise keine a-priori (Anfangswahrscheinlichkeit) Informationen der Themen $θ$  und Wortverteilungen $ϕ$ vorliegen, werden $α$ und $η$ symmetrisch bestimmt (Rieger at al. (2020 Juni), S. 120).

Ein hoher $η$ Wert führt zu einem Verlust der Gleichmäßigkeit der Wortmischung pro Thema, ein niedriger $η$ Wert erhöht und verbessert die Gleichmäßigkeit wodurch weniger dominantere Wörter pro Thema zugeordnet werden. Nach dem gleichen Prinzip steuert α die Mischung der Themen in den Texten (Rieger at al. (2020 Juni), S. 120).

LDA ist eine Weiterentwicklung von LSI, bzw. pLSI, wobei LSI selbst eine Weiterentwicklung von tf-idf ist (Blei et al. (2003), S. 994). LSI verbessert tf-idf indem eine größere reduktion der Beschreibungslänge in großen Daten möglich ist und die statistische Struktur zwischen oder innerhalb von Dokumenten offenbart (Blei et al. (2003), S. 994; Rosario, B. (2000)). Probability LSI (pLSI) verbessert LSI dahingehend, dass alle Worte in einem Dokument eine Stichprobe aus einem Mischungsmodell sind in welchem wiederum verschiedene Themen Multinomial-Verteilungen abgebildet sind (Blei et al. (2003), S. 994). Durch die linearität des Modelles kommt es zu overfitting, zusätzlich gibt es keine Möglichkeit nicht im Training enthaltenen Dokumenten eine Wahrscheinlichkeit zuzuweisen (Blei et al. (2003), S. 994).
Um Repräsentationen von Dokumenten und Wörtern austauschbar abzubilden, ist es notwendig mixture-models zu verwenden und somit den angeführten Limitationen von LSI und verwandten Ansätzen zu entgehen (Blei et al. (2003), S. 995).

Außerdem grenzt sich LDA von Hierarchical Latent Tree Analysis (HLTA) wie folgt ab, HLTA stellt Koinzidenz Muster explizit in Modellstrukturen dar (Chen at al. (2017), S. 1). HLTA ist somit eine Methode um Themen zu erkennen, indem ein Thema anhand von Wörtern identifiziert wird, die häufig in einem Thema und selten in einem anderen Thema auftreten (Liu et al. (2014), S. 1).

In [ ]:
import time
start_time = time.time()
lda_model = LdaModel(corpus=corpus, id2word=id2word,
                    num_topics=unique_categories,
                    random_state=42,
                    passes=10,
                    alpha="auto",
                    per_word_topics=True)
stop_time = time.time()
run_time = stop_time - start_time
print("Wall time:", run_time/60, "m")

In [32]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print("Coherence Score:", coherence_lda)

Coherence Score: 0.24897375347328105


In [33]:
data = {
    "Duration (minutes)": [run_time/60],
    "Coherence Score": [coherence_lda]
}
export_df = pd.DataFrame(data)
export_df.insert(0, 'Run', 1)
export_df.to_csv(r"data\vanilla_lda_data.csv", index=False)
export_df.to_csv('data/vanilla_lda_data.csv', index=False) # mac code
export_df.head()

,Run,Duration (minutes),Coherence Score
0,1,50.238107,0.248974


## PyLDAvis
### Step 5: Visualisierung mit pyLDAvis
Um zu visualisieren, wie unser LDA Model Wörter kategorisiert, nutzen wir die Library pyLDAvis.
<br>Mit ihr können wir uns einen interaktiven und intuitiven Überblick über unser Model verschaffen

In [35]:
import pyLDAvis
from pyLDAvis import gensim_models
pyLDAvis.enable_notebook()

In [36]:
visualisation = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [37]:
visualisation

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.384392 -0.277871       1        1  18.456422
21    -0.200793  0.375948       2        1   6.284229
20    -0.125548  0.097695       3        1   4.294225
14    -0.053486  0.026366       4        1   3.099291
9     -0.039119  0.017777       5        1   3.011122
18    -0.074282  0.041128       6        1   2.637397
2     -0.028838  0.012371       7        1   2.518690
30     0.050678 -0.015946       8        1   2.485874
19    -0.007089  0.002487       9        1   2.424251
26     0.023294 -0.008413      10        1   2.402057
34     0.003155 -0.001413      11        1   2.254002
13     0.042520 -0.013958      12        1   2.213180
7      0.003731 -0.001711      13        1   2.204422
22    -0.004279  0.001478      14        1   2.203084
8     -0.003871  0.001256      15        1   2.199256
24     0.010202 -0.004084      16        1   2.180925
4      0.003510 -0.001584      17        1   2.147843
28     0.014696 -0.005569      18        1   2.085967
16     0.030158 -0.010456      19        1   2.044277
3     -0.000722 -0.000001      20        1   2.043457
32     0.021185 -0.007688      21        1   2.030386
10     0.012259 -0.004697      22        1   2.015175
27     0.031990 -0.010949      23        1   1.997845
36     0.071017 -0.020549      24        1   1.976181
6      0.033288 -0.011393      25        1   1.935828
1      0.025929 -0.009212      26        1   1.927032
35     0.035387 -0.011932      27        1   1.906326
25     0.062332 -0.018704      28        1   1.859915
17     0.041755 -0.013666      29        1   1.830971
5      0.031251 -0.010721      30        1   1.800285
29     0.048062 -0.015316      31        1   1.761389
15     0.061571 -0.018563      32        1   1.725517
12     0.050013 -0.015844      33        1   1.687326
31     0.046066 -0.014764      34        1   1.680011
33     0.041883 -0.013727      35        1   1.634984
23     0.061398 -0.018450      36        1   1.579606
0      0.065090 -0.019326      37        1   1.461251, topic_info=         Term          Freq         Total Category  logprob  loglift
1564    photo  31539.000000  31539.000000  Default  30.0000  30.0000
145       day  22695.000000  22695.000000  Default  29.0000  29.0000
3310  wedding  10368.000000  10368.000000  Default  28.0000  28.0000
921      love  12123.000000  12123.000000  Default  27.0000  27.0000
518       way  14632.000000  14632.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
582     grand    488.968250    490.044856  Topic37  -4.6471   4.2237
4103  whitney    484.312850    485.389457  Topic37  -4.6567   4.2237
7507   period    483.883532    484.960138  Topic37  -4.6576   4.2237
174    aspect    446.027991    447.104597  Topic37  -4.7391   4.2235
2475  chronic    422.720839    423.797446  Topic37  -4.7927   4.2233

[1140 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
571      26  0.999445        1
212       2  0.999882       10
817      27  0.999346      100
590      13  0.999207       11
5244     14  0.999063       12
...     ...       ...      ...
2062      9  0.999449    youll
1575      3  0.999559    young
966       3  0.999811    youre
688      29  0.999488  youtube
7473      4  0.999570    youve

[1110 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 22, 21, 15, 10, 19, 3, 31, 20, 27, 35, 14, 8, 23, 9, 25, 5, 29, 17, 4, 33, 11, 28, 37, 7, 2, 36, 26, 18, 6, 30, 16, 13, 32, 34, 24, 1])

saving the pyldavis visual for later comparison in Benchmarks&Visualization (link)!

In [39]:
import pickle

In [40]:
with open('visuals/vanilla_lda.pkl', 'wb') as f:
    pickle.dump(visualisation, f)

### Step 6: Topics matchen und Prediction gegen Docs laufen lassen

In [59]:
def get_dominant_topics():
    """
        Gets the dominant topic for every row in the dataset
    """
    dominant_topics = []
    for doc in processed_texts:
        bow = id2word.doc2bow(doc)
        
        # Gets all the topics slightly dominant in the document
        topics = lda_model.get_document_topics(bow, minimum_probability=1/unique_categories)
        
        # Sorts the topics based off lowest probability and selects the last element
        dominant_topic, _ = sorted(topics, key=lambda x: x[1])[-1]
        dominant_topics.append(dominant_topic)
    return dominant_topics

dominant_topics = get_dominant_topics()
processed_df["dominant_topic"] = dominant_topics

C:\Users\JSCHNEE\AppData\Local\Temp\ipykernel_23152\1755556973.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df["dominant_topic"] = dominant_topics


In [61]:
def map_categories_to_topics(df):
    category_topic_mapping = {}
    for category in df["category"].unique():
        # Get all documents assigned to this category
        category_docs = df[df["category"] == category]
        
        # Get the most often predicted topic
        most_often_predicted = category_docs["dominant_topic"].mode()[0]
        category_topic_mapping[category] = most_often_predicted
    return category_topic_mapping

category_topic_mapping = map_categories_to_topics(processed_df)

In [63]:
pprint(category_topic_mapping)

{'ARTS': 11,
 'ARTS & CULTURE': 11,
 'BLACK VOICES': 11,
 'BUSINESS': 11,
 'COLLEGE': 11,
 'COMEDY': 11,
 'CRIME': 11,
 'CULTURE & ARTS': 11,
 'DIVORCE': 11,
 'EDUCATION': 11,
 'ENTERTAINMENT': 11,
 'ENVIRONMENT': 11,
 'FIFTY': 11,
 'FOOD & DRINK': 11,
 'GOOD NEWS': 11,
 'GREEN': 11,
 'HEALTHY LIVING': 11,
 'HOME & LIVING': 11,
 'IMPACT': 11,
 'LATINO VOICES': 11,
 'MEDIA': 11,
 'MONEY': 11,
 'PARENTING': 11,
 'PARENTS': 11,
 'POLITICS': 11,
 'QUEER VOICES': 11,
 'RELIGION': 11,
 'SCIENCE': 11,
 'SPORTS': 11,
 'STYLE': 11,
 'STYLE & BEAUTY': 11,
 'TASTE': 11,
 'TECH': 11,
 'TRAVEL': 11,
 'WEDDINGS': 11,
 'WELLNESS': 11,
 'WOMEN': 11}


Vielleicht war das hier der *falsche* Ansatz...

In [78]:
def map_topics_to_categories(processed_df):
    """
    Maps topics to the most common categories in the processed DataFrame.
    It does this by inspecting the predicted topics for all documents belonging to one category
    and picking the most often predicted topic as the category.

    Returns:
    dict: A dictionary mapping each topic to the most common category among the documents assigned to that topic.
    """
    topic_category_mapping = {}
    for topic in processed_df['dominant_topic'].unique():
        # Get documents assigned to the topic
        topic_docs = processed_df[processed_df['dominant_topic'] == topic]
        # Find the most common category among these documents
        if not topic_docs.empty:
            for category in topic_docs['category']:
                if category not in topic_category_mapping.values():
                    topic_category_mapping[topic] = category
    return topic_category_mapping

# Create the mapping
topic_category_mapping = map_topics_to_categories(processed_df)

In [80]:
pprint(topic_category_mapping)

{1: 'BUSINESS',
 2: 'IMPACT',
 3: 'POLITICS',
 6: 'COMEDY',
 7: 'ENTERTAINMENT',
 9: 'MONEY',
 10: 'DIVORCE',
 11: 'SPORTS',
 12: 'ENVIRONMENT',
 13: 'BLACK VOICES',
 14: 'FOOD & DRINK',
 16: 'MEDIA',
 18: 'STYLE & BEAUTY',
 19: 'TRAVEL',
 20: 'WELLNESS',
 21: 'PARENTING',
 22: 'PARENTS',
 26: 'CRIME',
 27: 'GREEN',
 28: 'HOME & LIVING',
 29: 'WEDDINGS',
 30: 'RELIGION',
 36: 'QUEER VOICES'}


In [82]:
def assign_predicted_category(processed_df, topic_category_mapping):
    """
    Assigns predicted categories to the processed DataFrame based on the dominant topic.

    Parameters:
    processed_df (pandas.DataFrame): The DataFrame containing processed text and dominant topics.
    topic_category_mapping (dict): A dictionary mapping each topic to the most common category among the documents assigned to that topic.

    Returns:
    pandas.DataFrame: The updated DataFrame with a new column 'predicted_category' containing the assigned categories.
    """
    processed_df['predicted_category'] = processed_df['dominant_topic'].map(topic_category_mapping)
    return processed_df

# Assign predicted categories
processed_df = assign_predicted_category(processed_df, topic_category_mapping)


C:\Users\JSCHNEE\AppData\Local\Temp\ipykernel_23152\600335101.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['predicted_category'] = processed_df['dominant_topic'].map(topic_category_mapping)


In [86]:
import pandas as pd

# Ensure the DataFrame `processed_df` has 'category' and 'predicted_category' columns
# Create a new column to indicate matches
processed_df['match'] = processed_df['category'] == processed_df['predicted_category']

# Group by category and count matches
match_counts = processed_df.groupby('category')['match'].sum().reset_index()
match_counts.columns = ['category', 'match_count']

# Count total entries per category
category_counts = processed_df['category'].value_counts(normalize=False).reset_index()
category_counts.columns = ['category', 'total_count']

# Merge match counts with total counts
comparison = pd.merge(match_counts, category_counts, on='category')

# Calculate match percentage
comparison['match_percentage'] = (comparison['match_count'] / comparison['total_count']) * 100

# Split into matched and no-match categories
matched_categories = comparison[comparison['match_count'] > 0]
no_match_categories = comparison[comparison['match_count'] == 0]

# Sort matched categories by match percentage (descending)
matched_categories_sorted = matched_categories.sort_values(by='match_percentage', ascending=False)

# Save the results as CSV
matched_categories_sorted.to_csv('data/vanilla-matched_categories.csv', index=False)
no_match_categories.to_csv('data/vanilla-no_match_categories.csv', index=False)

# Return or print the data
print(matched_categories_sorted)
print(no_match_categories)

          category  match_count  total_count  match_percentage
28          SPORTS         5057         5077         99.606067
2     BLACK VOICES           14         4583          0.305477
22       PARENTING           17         8791          0.193380
11     ENVIRONMENT            2         1444          0.138504
26        RELIGION            2         2577          0.077610
15           GREEN            2         2622          0.076278
21           MONEY            1         1756          0.056948
6            CRIME            2         3562          0.056148
34        WEDDINGS            2         3653          0.054750
20           MEDIA            1         2943          0.033979
30  STYLE & BEAUTY            3         9814          0.030569
8          DIVORCE            1         3426          0.029189
18          IMPACT            1         3484          0.028703
23         PARENTS            1         3955          0.025284
17   HOME & LIVING            1         4320          0

C:\Users\JSCHNEE\AppData\Local\Temp\ipykernel_23152\71953698.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['match'] = processed_df['category'] == processed_df['predicted_category']


In [88]:
# Prepare a DataFrame with the required columns
results_df = processed_df[['headline', 'short_description', 'category', 'predicted_category']]

# Save this DataFrame as CSV
results_df.to_csv('data/vanilla-predicted_categories.csv', index=False)

# Return or print the results
print(results_df.head(10))  # Show the first 10 rows for visual confirmation

                                             headline  \
2   23 Of The Funniest Tweets About Cats And Dogs ...   
3   The Funniest Tweets From Parents This Week (Se...   
8   How A New Documentary Captures The Complexity ...   
13  Twitch Bans Gambling Sites After Streamer Scam...   
16  'Reboot' Is A Clever And Not Too Navel-Gazey L...   
17  Maury Wills, Base-Stealing Shortstop For Dodge...   
20  Golden Globes Returning To NBC In January Afte...   
21  Biden Says U.S. Forces Would Defend Taiwan If ...   
24  ‘Beautiful And Sad At The Same Time’: Ukrainia...   
26  Las Vegas Aces Win First WNBA Title, Chelsea G...   

                                    short_description        category  \
2   "Until you have a dog you don't understand wha...          COMEDY   
3   "Accidentally put grown-up toothpaste on my to...       PARENTING   
8   In "Mija," director Isabel Castro combined mus...  CULTURE & ARTS   
13  One man's claims that he scammed people on the...            TECH   
16  Sta

# <a href="https://github.com/NiklasElsaesser/bug-free-fishstick/blob/main/Improved_LDA.ipynb">Improved_LDA</a>

# <a href="https://github.com/NiklasElsaesser/bug-free-fishstick/blob/main/Benchmarks%26Visualizations.ipynb">Benchmarks & Visualizations</a>

# Literaturhinweise
* Rosario, B. (2000). Latent semantic indexing: An overview. Techn. rep. INFOSYS, 240, 1-16.
* Chen, P., Zhang, N. L., Liu, T., Poon, L. K., Chen, Z., & Khawar, F. (2017). Latent tree models for hierarchical topic detection. Artificial Intelligence, 250, 105-124.
* Liu, T., Zhang, N. L., & Chen, P. (2014). Hierarchical latent tree analysis for topic detection. In Machine Learning and Knowledge Discovery in Databases: European Conference, ECML PKDD 2014, Nancy, France, September 15-19, 2014. Proceedings, Part II 14 (pp. 256-272). Springer Berlin Heidelberg.
* Rieger, J., Rahnenführer, J., & Jentsch, C. (2020, June). Improving latent Dirichlet allocation: On reliability of the novel method LDAPrototype. In International conference on applications of natural language to information systems (pp. 118-125). Cham: Springer International Publishing.
* Blei, D. M., Ng, A. Y., & Jordan, M. I. (2003). Latent dirichlet allocation. Journal of machine Learning research, 3(Jan), 993-1022.
* Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-265). Cambridge: Cambridge University Press.
* Rieger, J., Koppers, L., Jentsch, C., Rahnenfu ̈hrer, J.: Improving Reliability of Latent Dirichlet Allocation by Assessing Its Stability Using Clustering Techniques on Replicated Runs (2020)
* Heyer, Gerhard, Wiedemann, Gregor, Niekler, Andreas (2020): Topic-Modelle und ihr Potenzial für die philologische Forschung, In Lobin, Henning, Schneider, Roman, Witt, Andreas (Hrsg.): Digitale Infrastrukturen für die germanistische Forschung (S.351-353), de Gruyter, 23.07.2018.
* Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).